# Connecting to DB in RDS Guide
### Miles Child - November 2022
#### Northeastern Trading Systems

#### Starting DB and importing necessary packages

Once you have created your database in RDS, make sure that it is configured to be publicly accessible and that you've attached a security group that allows inbound/outbound access from 'All Traffic' on both IPv4 and IPv6. Save the login credentials of the DB as soon as you launch the instance.

Next, navigate to your local python environment and install the pymysql package. This process can be very messy if done incorrectly the first time, so make sure you do the following:

For Python 2.x:
- sudo pip install pymysql.

For Python 3.x:
- sudo pip3 install pymysql.

For either running on Anaconda:
- sudo conda install pymysql.

If that didn't work try APT:
- sudo apt-get install pymysql.

credit: https://stackoverflow.com/a/49354387/20355075

Finally, 'import pymysql' in your project before continuing.

In [1]:
# imports
import pymysql
from config import *  # credentials stored here

Now, establish a connection to the db by using pymysql.connect(host=<'someurl.rds.amazonaws.com'>, user=<'username'>, password=<'password'>).

Create a cursor, which can essentially be regarded as an object that executes your queries (written locally) on the DB, stores the result in-memory, and returns it back to this project. Check that the connection and cursor have been made properly by checking the version of the db.

In [2]:
connection = pymysql.connect(host=RDS_HOSTNAME, user=RDS_USER, password=RDS_PASSWORD)
cursor = connection.cursor()
cursor.execute('select version()')

1

After successfully establishing a connection to the RDS instance, create a DB and add the neccessary tables/entities/attributes.

In [8]:
# Create a new database
cursor.execute("CREATE DATABASE nts_lightning_db;")
cursor.execute(f"CREATE USER 'webapp'@'%' IDENTIFIED BY {DOCKER_DB_PASSWORD};")
cursor.execute("GRANT ALL PRIVILEGES ON nts_lightning_db.* TO 'webapp'@'%';")
cursor.execute("FLUSH PRIVILEGES;")
cursor.execute("USE nts_lightning_db;")

ProgrammingError: (1007, "Can't create database 'nts_lightning_db'; database exists")

In [9]:
# Create the tables
cursor.execute("""CREATE TABLE strategy (
    strategy_name varchar(50),
    strategy_id integer AUTO_INCREMENT NOT NULL,
    documentation_link varchar(50),
    launch_date datetime,
    termination_date datetime,
    PRIMARY KEY (strategy_id)
);""")

OperationalError: (1050, "Table 'strategy' already exists")

In [13]:
cursor.execute("""CREATE TABLE trade (
    trade_id integer UNIQUE NOT NULL,
    strategy_id integer,
    open_time datetime,
    close_time datetime,
    PRIMARY KEY (trade_id),
    CONSTRAINT fk_1 
        FOREIGN KEY (strategy_id) REFERENCES strategy(strategy_id)
        ON UPDATE cascade ON DELETE restrict
);""")

0

In [15]:
cursor.execute("""CREATE TABLE trade_leg (
    leg_no integer NOT NULL,
    trade_id integer NOT NULL,
    contract varchar(50),
    open_time datetime,
    close_time datetime,
    PRIMARY KEY(leg_no, trade_id),
    CONSTRAINT fk_2
        FOREIGN KEY (trade_id) REFERENCES trade (trade_id)
        ON UPDATE restrict ON DELETE restrict
);""")

0

In [18]:
cursor.execute("""CREATE TABLE fill (
    fill_id integer NOT NULL,
    leg_no integer NOT NULL,
    trade_id integer NOT NULL,
    contract varchar(50),
    qty integer,
    avg float(32),
    placement_time datetime,
    filled_time datetime,
    PRIMARY KEY (fill_id),
    CONSTRAINT fk_3
        FOREIGN KEY (leg_no, trade_id) REFERENCES trade_leg (leg_no, trade_id)
        ON UPDATE cascade ON DELETE restrict
);""")

0

After creating the database architecture, insert testing (or historical) data into the tables.

note: mockaroo is good for generating dummy data https://www.mockaroo.com/

In [10]:
# Strategy Dummy Data
cursor.execute("""insert into strategy (strategy_name, strategy_id, documentation_link, launch_date, termination_date) 
values 
('MeanReversionLunchBreak', '1', 'https://github.com/QuantConnect/Lean/blob/master/Algorithm.Python/Alphas/MeanReversionLunchBreakAlpha.py',
 '2022-11-20 00:00:00', NULL);""")


1

In [11]:
# Trade Dummy Data
cursor.execute("""insert into trade (trade_id, strategy_id, open_time, close_time) 
values 
('000000001', '1', '2022-11-21 12:00:00', '2022-11-21 12:25:00');
""")
cursor.execute("""insert into trade (trade_id, strategy_id, open_time, close_time) 
values 
('000000002', '1', '2022-11-22 11:49:00', '2022-11-22 12:03:00');
""")

1

In [12]:
# Trade Leg Dummy Data
cursor.execute("""insert into trade_leg (leg_no, trade_id, contract, open_time, close_time) 
values 
('1', '000000001', 'SPY', '2022-11-21 12:00:00', '2022-11-21 12:25:00');
""")

# note: the below data represents the following trade:
# Seller of AAPL 148/150 Dec. 02 call spread
cursor.execute("""insert into trade_leg (leg_no, trade_id, contract, open_time, close_time) 
values 
('1', '000000002', 'AAPL221202C00148000', '2022-11-22 11:49:00', '2022-11-22 12:03:00');
""")
cursor.execute("""insert into trade_leg (leg_no, trade_id, contract, open_time, close_time) 
values 
('2', '000000002', 'AAPL221202C00150000', '2022-11-22 11:49:00', '2022-11-22 12:03:00');
""")

1

In [13]:
# Fill Dummy Data
cursor.execute("""insert into fill (fill_id, leg_no, trade_id, contract, qty, avg, placement_time, filled_time) 
values 
('000000001', '1', '000000001', 'SPY', '-100', '400.00', '2022-11-21 12:00:00', '2022-11-21 12:00:01');
""")

cursor.execute("""insert into fill (fill_id, leg_no, trade_id, contract, qty, avg, placement_time, filled_time) 
values 
('000000002', '1', '000000001', 'SPY', '100', '397.00', '2022-11-21 12:24:58', '2022-11-21 12:25:00');
""")


# note: the below data represents the following trade:
# Seller of AAPL 148/150 Dec. 02 call spread 2.50 @ 2.00, 10 x 10

cursor.execute("""insert into fill (fill_id, leg_no, trade_id, contract, qty, avg, placement_time, filled_time) 
values 
('000000003', '1', '000000002', 'AAPL221202C00148000', '-10', '2.50', '2022-11-22 11:49:00', '2022-11-22 11:49:01');
""")

cursor.execute("""insert into fill (fill_id, leg_no, trade_id, contract, qty, avg, placement_time, filled_time) 
values 
('000000004', '2', '000000002', 'AAPL221202C00150000', '10', '2.00', '2022-11-21 11:49:00', '2022-11-21 11:49:01');
""")

cursor.execute("""insert into fill (fill_id, leg_no, trade_id, contract, qty, avg, placement_time, filled_time) 
values 
('000000005', '1', '000000002', 'AAPL221202C00148000', '10', '2.25', '2022-11-22 12:02:59', '2022-11-22 12:02:59');
""")

cursor.execute("""insert into fill (fill_id, leg_no, trade_id, contract, qty, avg, placement_time, filled_time) 
values 
('000000006', '2', '000000002', 'AAPL221202C00150000', '-10', '1.90', '2022-11-22 12:02:59', '2022-11-22 12:03:00');
""")

1

In [14]:
# Testing that all dummy data was inserted properly and exists in the db.

cursor.execute('SELECT * from fill;')
col_headers = [x[0] for x in cursor.description]
json_data = []
the_data = cursor.fetchall()
for row in the_data:
    json_data.append(dict(zip(col_headers, row)))
json_data

[{'fill_id': 1,
  'leg_no': 1,
  'trade_id': 1,
  'contract': 'SPY',
  'qty': -100,
  'avg': 400.0,
  'placement_time': datetime.datetime(2022, 11, 21, 12, 0),
  'filled_time': datetime.datetime(2022, 11, 21, 12, 0, 1)},
 {'fill_id': 2,
  'leg_no': 1,
  'trade_id': 1,
  'contract': 'SPY',
  'qty': 100,
  'avg': 397.0,
  'placement_time': datetime.datetime(2022, 11, 21, 12, 24, 58),
  'filled_time': datetime.datetime(2022, 11, 21, 12, 25)},
 {'fill_id': 3,
  'leg_no': 1,
  'trade_id': 2,
  'contract': 'AAPL221202C00148000',
  'qty': -10,
  'avg': 2.5,
  'placement_time': datetime.datetime(2022, 11, 22, 11, 49),
  'filled_time': datetime.datetime(2022, 11, 22, 11, 49, 1)},
 {'fill_id': 4,
  'leg_no': 2,
  'trade_id': 2,
  'contract': 'AAPL221202C00150000',
  'qty': 10,
  'avg': 2.0,
  'placement_time': datetime.datetime(2022, 11, 21, 11, 49),
  'filled_time': datetime.datetime(2022, 11, 21, 11, 49, 1)},
 {'fill_id': 5,
  'leg_no': 1,
  'trade_id': 2,
  'contract': 'AAPL221202C00148000',


In [22]:
# Testing get_strategy_info

import pandas as pd

cursor.execute('SELECT * from strategy;')
col_headers = [x[0] for x in cursor.description]
json_data = []
the_data = cursor.fetchall()
for row in the_data:
    json_data.append(dict(zip(col_headers, row)))
json_data

[{'strategy_name': 'MeanReversionLunchBreak',
  'strategy_id': 1,
  'documentation_link': 'https://github.com/QuantConnect/Lean/blob/master/A',
  'launch_date': datetime.datetime(2022, 11, 20, 0, 0),
  'termination_date': None}]

In [30]:
# Testing get_trade_info

# TODO: FIX QUERY

cursor.execute("""
SELECT * from trade join trade_leg on trade.trade_id = trade_leg.trade_id join fill on trade_leg.leg_no = fill.leg_no and trade_leg.trade_id = fill.trade_id
where trade.trade_id = '000000002'
order by trade_leg.leg_no, fill.placement_time;
""")
col_headers = [x[0] for x in cursor.description]
json_data = []
the_data = cursor.fetchall()
for row in the_data:
    json_data.append(dict(zip(col_headers, row)))
df = pd.DataFrame(json_data)
df


,trade_id,strategy_id,open_time,close_time,leg_no,contract,fill_id,qty,avg,placement_time,filled_time
0,2,1,2022-11-22 11:49:00,2022-11-22 12:03:00,1,AAPL221202C00148000,3,-10,2.50,2022-11-22 11:49:00,2022-11-22 11:49:01
1,2,1,2022-11-22 11:49:00,2022-11-22 12:03:00,1,AAPL221202C00148000,5,10,2.25,2022-11-22 12:02:59,2022-11-22 12:02:59
2,2,1,2022-11-22 11:49:00,2022-11-22 12:03:00,2,AAPL221202C00150000,4,10,2.00,2022-11-21 11:49:00,2022-11-21 11:49:01
3,2,1,2022-11-22 11:49:00,2022-11-22 12:03:00,2,AAPL221202C00150000,6,-10,1.90,2022-11-22 12:02:59,2022-11-22 12:03:00


In [34]:
# Testing get_trade_leg_info
cursor.execute("""
Select * from trade_leg join fill on trade_leg.leg_no = fill.leg_no and trade_leg.trade_id = fill.trade_id
where trade_leg.trade_id = '000000002' and trade_leg.leg_no = '1'
order by fill.placement_time;
""")
col_headers = [x[0] for x in cursor.description]
json_data = []
the_data = cursor.fetchall()
for row in the_data:
    json_data.append(dict(zip(col_headers, row)))
df = pd.DataFrame(json_data)
df

,leg_no,trade_id,contract,open_time,close_time,fill_id,qty,avg,placement_time,filled_time
0,1,2,AAPL221202C00148000,2022-11-22 11:49:00,2022-11-22 12:03:00,3,-10,2.50,2022-11-22 11:49:00,2022-11-22 11:49:01
1,1,2,AAPL221202C00148000,2022-11-22 11:49:00,2022-11-22 12:03:00,5,10,2.25,2022-11-22 12:02:59,2022-11-22 12:02:59


In [35]:
# Testing get_fill_info
cursor.execute("""
Select * from fill where fill.fill_id = '000000002';
""")
col_headers = [x[0] for x in cursor.description]
json_data = []
the_data = cursor.fetchall()
for row in the_data:
    json_data.append(dict(zip(col_headers, row)))
json_data
#df = pd.DataFrame(json_data)
#df

[{'fill_id': 2,
  'leg_no': 1,
  'trade_id': 1,
  'contract': 'SPY',
  'qty': 100,
  'avg': 397.0,
  'placement_time': datetime.datetime(2022, 11, 21, 12, 24, 58),
  'filled_time': datetime.datetime(2022, 11, 21, 12, 25)}]

In [36]:
# Testing get_open_trades
strategy = 'MeanReversionLunchBreak'
cursor.execute(f"""
Select * from strategy 
join trade on strategy.strategy_id = trade.strategy_id 
join trade_leg on trade.trade_id = trade_leg.trade_id 
join fill on trade_leg.leg_no = fill.leg_no and trade_leg.trade_id = fill.trade_id
where strategy.strategy_name = '{strategy}' and trade_leg.close_time is null
order by fill.placement_time;
""")
col_headers = [x[0] for x in cursor.description]
json_data = []
the_data = cursor.fetchall()
for row in the_data:
    json_data.append(dict(zip(col_headers, row)))
json_data

[]

In [21]:
# Testing get_historical_trades